# Resource Allocation Code Demo

The Food Bank of the Southern Tier (FBST) is a member of Feeding America, focused on providing food security for people with limited financial resources, and serves six counties and nearly 4,000 square miles in the New York.  Under normal operations (non COVID times), the Mobile Food Pantry program is among the main activities of the FBST.  The goal of the service is to make nutritious and healthy food more accessible to people in underserved communities.  Even in areas where other agencies provide assistance, clients may not always have access to food due to limited public transportation options, or because those agencies are only open hours or days per work.

Here we do a sample experiment testing out some of the existing and developed algorithms against a randomized heuristic.

In [1]:
import or_suite
import numpy as np

import copy

import os
from stable_baselines3.common.monitor import Monitor
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
import pandas as pd


import gym

In [2]:
# Getting out configuration parameter for the environment
CONFIG =  or_suite.envs.env_configs.resource_allocation_default_config


# Specifying training iteration, epLen, number of episodes, and number of iterations
epLen = CONFIG['num_rounds']
nEps = 1
numIters = 25

# Configuration parameters for running the experiment
DEFAULT_SETTINGS = {'seed': 1, 
                    'recFreq': 1, 
                    'dirPath': '../data/resource/', 
                    'deBug': False, 
                    'nEps': nEps, 
                    'numIters': numIters, 
                    'saveTrajectory': True, # save trajectory for calculating additional metrics
                    'epLen' : epLen,
                    'render': False,
                    'pickle': False # indicator for pickling final information
                    }

resource_env = gym.make('Resource-v0', config=CONFIG)
mon_env = Monitor(resource_env)


In [3]:
agents = { # 'SB PPO': PPO(MlpPolicy, mon_env, gamma=1, verbose=0, n_steps=epLen),
# 'Random': or_suite.agents.rl.random.randomAgent(),
# 'Equal': or_suite.agents.resource_allocation.equal_allocation.equalAllocationAgent(epLen, CONFIG),
# 'FixedThreshold': or_suite.agents.resource_allocation.fixed_threshold.fixedThresholdAgent(epLen, CONFIG),
'Guardrail-0.5': or_suite.agents.resource_allocation.hope_guardrail.hopeguardrailAgent(epLen, CONFIG, 0.5),
'Guardrail-0.3': or_suite.agents.resource_allocation.hope_guardrail.hopeguardrailAgent(epLen, CONFIG, 0.3),
'Guardrail-0.25': or_suite.agents.resource_allocation.hope_guardrail.hopeguardrailAgent(epLen, CONFIG, 0.25)
}

Mean and variance endomwnets:
[[2.016 2.005 2.041 1.953 2.049 1.941 2.034 2.025 1.991 1.976]
 [2.952 3.073 3.042 2.964 3.06  2.967 2.956 2.999 2.956 3.004]
 [4.017 3.96  3.948 4.07  3.975 4.032 4.046 3.995 3.921 3.953]] [[1.033744 1.012975 1.117319 0.914791 0.984599 0.989519 0.992844 1.088375
  0.964919 0.957424]
 [1.809696 2.083671 2.146236 2.028704 2.0164   1.933911 1.900064 1.980999
  2.034064 1.895984]
 [3.106711 2.9084   2.979296 3.1871   2.964375 2.966976 3.115884 3.168975
  2.814759 3.026791]]
Mean and variance endomwnets:
[[1.979 1.961 2.004 1.953 1.928 2.048 1.997 2.016 1.97  1.982]
 [2.986 2.972 3.011 2.964 3.019 2.984 3.051 3.016 2.994 2.972]
 [3.934 4.006 4.115 3.866 4.04  4.058 3.971 4.032 4.05  4.066]] [[1.084559 0.963479 0.979984 0.968791 0.932816 1.023696 1.046991 1.021744
  0.8931   1.071676]
 [1.961804 1.877216 1.998879 1.960704 1.950639 1.919744 2.080399 2.007744
  2.085964 2.077216]
 [2.803644 3.011964 3.141775 2.740044 2.9744   2.892636 2.862159 2.938976
  2.9055  

# Step 5: Run Simulations

Run the different heuristics in the environment

In [4]:
import warnings
warnings.simplefilter('ignore')

In [5]:
path_list_line = []
algo_list_line = []
path_list_radar = []
algo_list_radar= []
for agent in agents:
    print(agent)
    DEFAULT_SETTINGS['dirPath'] = '../data/resource_'+str(agent)+'/'
    if agent == 'SB PPO':
        or_suite.utils.run_single_sb_algo(mon_env, agents[agent], DEFAULT_SETTINGS)
    elif agent == 'AdaQL' or agent == 'Unif QL' or agent == 'AdaMB' or agent == 'Unif MB':
        or_suite.utils.run_single_algo_tune(resource_env, agents[agent], scaling_list, DEFAULT_SETTINGS)
    else:
        or_suite.utils.run_single_algo(resource_env, agents[agent], DEFAULT_SETTINGS)

    path_list_line.append('../data/resource_'+str(agent))
    algo_list_line.append(str(agent))
    if agent != 'SB PPO':
        path_list_radar.append('../data/resource_'+str(agent)+'/')
        algo_list_radar.append(str(agent))     
        
fig_path = '../figures/'
fig_name = 'resource'+'_line_plot'+'.pdf'
or_suite.plots.plot_line_plots(path_list_line, algo_list_line, fig_path, fig_name, int(nEps / 40)+1)        
        
additional_metric = {'Efficiency': lambda traj : or_suite.utils.delta_EFFICIENCY(traj, CONFIG), \
                      'Hindsight Envy': lambda traj : or_suite.utils.delta_HINDSIGHT_ENVY(traj, CONFIG), \
                      'Counterfactual Envy': lambda traj : or_suite.utils.delta_COUNTERFACTUAL_ENVY(traj, CONFIG)}
#                       'Prop': lambda traj : or_suite.utils.delta_PROP(traj, CONFIG), \
#                       'Exante Envy': lambda traj : or_suite.utils.delta_EXANTE_ENVY(traj, CONFIG)}
fig_name = 'resource'+'_radar_plot'+'.pdf'
or_suite.plots.plot_radar_plots(path_list_radar, algo_list_radar,
fig_path, fig_name,
additional_metric
)

Guardrail-0.5
Lower and Upper Solutions:
[[0.      0.18155]
 [0.      0.18155]
 [0.21175 0.     ]]
[[0.      0.26552]
 [0.      0.26552]
 [0.30968 0.     ]]
uf 0.36309999227523804
logged -1.0130770218363225
uf 1.6339499652385712
logged 0.49100037493995163
uf 0.21175000071525574
logged -1.5523489420721421
-1.0769727425111717
uf 0.36309999227523804
logged -1.0130770218363225
uf 1.6339499652385712
logged 0.49100037493995163
uf 0.21175000071525574
logged -1.5523489420721421
-0.9279703105753957
uf 0.36309999227523804
logged -1.0130770218363225
uf 1.6339499652385712
logged 0.49100037493995163
uf 0.21175000071525574
logged -1.5523489420721421
-0.9741026227896412
uf 0.36309999227523804
logged -1.0130770218363225
uf 1.6339499652385712
logged 0.49100037493995163
uf 0.21175000071525574
logged -1.5523489420721421
-0.6271548312201407
uf 0.36309999227523804
logged -1.0130770218363225
uf 1.6339499652385712
logged 0.49100037493995163
uf 0.21175000071525574
logged -1.5523489420721421
-1.027294317327720

logged 0.49100037493995163
uf 0.21175000071525574
logged -1.5523489420721421
-0.6271548312201407
uf 0.36309999227523804
logged -1.0130770218363225
uf 1.6339499652385712
logged 0.49100037493995163
uf 0.3096800148487091
logged -1.1722157247641887
-0.2875614406028298
uf 0.36309999227523804
logged -1.0130770218363225
uf 1.6339499652385712
logged 0.49100037493995163
uf 0.21175000071525574
logged -1.5523489420721421
-0.8314897629213501
uf 0.36309999227523804
logged -1.0130770218363225
uf 1.6339499652385712
logged 0.49100037493995163
uf 0.3096800148487091
logged -1.1722157247641887
-0.3087799343265453
uf 0.36309999227523804
logged -1.0130770218363225
uf 1.6339499652385712
logged 0.49100037493995163
uf 0.3096800148487091
logged -1.1722157247641887
-0.4835514701309451
uf 0.36309999227523804
logged -1.0130770218363225
uf 1.6339499652385712
logged 0.49100037493995163
uf 0.21175000071525574
logged -1.5523489420721421
-1.0947033073571981
uf 0.36309999227523804
logged -1.0130770218363225
uf 1.633949

uf 1.6339499652385712
logged 0.49100037493995163
uf 0.21175000071525574
logged -1.5523489420721421
-0.5732276391965588
uf 0.36309999227523804
logged -1.0130770218363225
uf 1.6339499652385712
logged 0.49100037493995163
uf 0.21175000071525574
logged -1.5523489420721421
-1.0769727425111717
uf 0.36309999227523804
logged -1.0130770218363225
uf 1.6339499652385712
logged 0.49100037493995163
uf 0.21175000071525574
logged -1.5523489420721421
-0.017251991496453976
uf 0.36309999227523804
logged -1.0130770218363225
uf 1.6339499652385712
logged 0.49100037493995163
uf 0.21175000071525574
logged -1.5523489420721421
-0.9741026227896412
uf 0.36309999227523804
logged -1.0130770218363225
uf 1.6339499652385712
logged 0.49100037493995163
uf 0.21175000071525574
logged -1.5523489420721421
-0.599588960461842
uf 0.36309999227523804
logged -1.0130770218363225
uf 1.6339499652385712
logged 0.49100037493995163
uf 0.21175000071525574
logged -1.5523489420721421
-1.0358246946225593
uf 0.36309999227523804
logged -1.01

logged -0.6329179068381764
uf 2.3896800577640533
logged 0.8711594899380977
uf 0.21175000071525574
logged -1.5523489420721421
-0.251007889715192
uf 0.5310400128364563
logged -0.6329179068381764
uf 2.3896800577640533
logged 0.8711594899380977
uf 0.21175000071525574
logged -1.5523489420721421
-0.3990593622212531
uf 0.5310400128364563
logged -0.6329179068381764
uf 2.3896800577640533
logged 0.8711594899380977
uf 0.21175000071525574
logged -1.5523489420721421
-0.7166140752610908
uf 0.5310400128364563
logged -0.6329179068381764
uf 2.3896800577640533
logged 0.8711594899380977
uf 0.21175000071525574
logged -1.5523489420721421
-0.5912742921964012
uf 0.36309999227523804
logged -1.0130770218363225
uf 1.6339499652385712
logged 0.49100037493995163
uf 0.21175000071525574
logged -1.5523489420721421
-1.0769727425111717
uf 0.36309999227523804
logged -1.0130770218363225
uf 1.6339499652385712
logged 0.49100037493995163
uf 0.21175000071525574
logged -1.5523489420721421
-0.9066936327601638
uf 0.363099992275

logged -1.0089544091521232
uf 1.6407000124454498
logged 0.49512298762415086
uf 0.21030999720096588
logged -1.5591726595956374
-0.5215550249461526
uf 0.3646000027656555
logged -1.0089544091521232
uf 1.6407000124454498
logged 0.49512298762415086
uf 0.21030999720096588
logged -1.5591726595956374
-0.9192564322878465
uf 0.3646000027656555
logged -1.0089544091521232
uf 1.6407000124454498
logged 0.49512298762415086
uf 0.21030999720096588
logged -1.5591726595956374
-0.9803917934482368
uf 0.3646000027656555
logged -1.0089544091521232
uf 1.6407000124454498
logged 0.49512298762415086
uf 0.21030999720096588
logged -1.5591726595956374
-0.7200345105827776
uf 0.3646000027656555
logged -1.0089544091521232
uf 1.6407000124454498
logged 0.49512298762415086
uf 0.21030999720096588
logged -1.5591726595956374
-0.8150258316918367
uf 0.3646000027656555
logged -1.0089544091521232
uf 1.6407000124454498
logged 0.49512298762415086
uf 0.21030999720096588
logged -1.5591726595956374
-0.8150258316918367
uf 0.364600002

uf 0.3646000027656555
logged -1.0089544091521232
uf 1.6407000124454498
logged 0.49512298762415086
uf 0.21030999720096588
logged -1.5591726595956374
-0.752136721534927
uf 0.3646000027656555
logged -1.0089544091521232
uf 1.6407000124454498
logged 0.49512298762415086
uf 0.21030999720096588
logged -1.5591726595956374
-1.1084966384375208
uf 0.3646000027656555
logged -1.0089544091521232
uf 1.6407000124454498
logged 0.49512298762415086
uf 0.4216200113296509
logged -0.8636508176782649
-0.34359215460888315
uf 0.3646000027656555
logged -1.0089544091521232
uf 1.6407000124454498
logged 0.49512298762415086
uf 0.21030999720096588
logged -1.5591726595956374
-1.0792354891147942
uf 0.3646000027656555
logged -1.0089544091521232
uf 1.6407000124454498
logged 0.49512298762415086
uf 0.21030999720096588
logged -1.5591726595956374
-0.7827044021151224
uf 0.3646000027656555
logged -1.0089544091521232
uf 1.6407000124454498
logged 0.49512298762415086
uf 0.21030999720096588
logged -1.5591726595956374
-1.1648311410

logged -1.5591726595956374
-1.0792354891147942
uf 0.3646000027656555
logged -1.0089544091521232
uf 1.6407000124454498
logged 0.49512298762415086
uf 0.21030999720096588
logged -1.5591726595956374
-0.30668682540668496
uf 0.3646000027656555
logged -1.0089544091521232
uf 1.6407000124454498
logged 0.49512298762415086
uf 0.21030999720096588
logged -1.5591726595956374
-0.9997472269203974
uf 0.3646000027656555
logged -1.0089544091521232
uf 1.6407000124454498
logged 0.49512298762415086
uf 0.21030999720096588
logged -1.5591726595956374
-1.2336084223877248
uf 0.3646000027656555
logged -1.0089544091521232
uf 1.6407000124454498
logged 0.49512298762415086
uf 0.21030999720096588
logged -1.5591726595956374
-1.2086468653615468
uf 0.7309399843215942
logged -0.3134239233940657
uf 3.289229929447174
logged 1.1906534733822083
uf 0.21030999720096588
logged -1.5591726595956374
-0.8096836593559416
uf 0.3646000027656555
logged -1.0089544091521232
uf 1.6407000124454498
logged 0.49512298762415086
uf 0.21030999720

uf 1.6424100548028946
logged 0.49616470872217083
uf 0.21201999485492706
logged -1.5510746934041084
-0.8446015822742126
uf 0.364980012178421
logged -1.0079126880541032
uf 1.6424100548028946
logged 0.49616470872217083
uf 0.21201999485492706
logged -1.5510746934041084
-0.6475694162692525
uf 0.364980012178421
logged -1.0079126880541032
uf 1.6424100548028946
logged 0.49616470872217083
uf 0.21201999485492706
logged -1.5510746934041084
-1.29203455265648
uf 0.364980012178421
logged -1.0079126880541032
uf 1.6424100548028946
logged 0.49616470872217083
uf 0.21201999485492706
logged -1.5510746934041084
-0.8109600055537415
uf 0.364980012178421
logged -1.0079126880541032
uf 1.6424100548028946
logged 0.49616470872217083
uf 0.21201999485492706
logged -1.5510746934041084
-0.5217546005557987
uf 0.364980012178421
logged -1.0079126880541032
uf 1.6424100548028946
logged 0.49616470872217083
uf 0.21201999485492706
logged -1.5510746934041084
-0.6778627320185961
uf 0.364980012178421
logged -1.0079126880541032


uf 1.6424100548028946
logged 0.49616470872217083
uf 0.21201999485492706
logged -1.5510746934041084
-0.8939799463969405
uf 0.364980012178421
logged -1.0079126880541032
uf 1.6424100548028946
logged 0.49616470872217083
uf 0.21201999485492706
logged -1.5510746934041084
-0.5692303309485952
uf 0.364980012178421
logged -1.0079126880541032
uf 1.6424100548028946
logged 0.49616470872217083
uf 0.21201999485492706
logged -1.5510746934041084
-1.0740776250455604
uf 0.364980012178421
logged -1.0079126880541032
uf 1.6424100548028946
logged 0.49616470872217083
uf 0.21201999485492706
logged -1.5510746934041084
-0.012794945916813941
uf 0.364980012178421
logged -1.0079126880541032
uf 1.6424100548028946
logged 0.49616470872217083
uf 0.21201999485492706
logged -1.5510746934041084
-0.9713695922037879
uf 0.364980012178421
logged -1.0079126880541032
uf 1.6424100548028946
logged 0.49616470872217083
uf 0.21201999485492706
logged -1.5510746934041084
-0.5960918060142737
uf 0.364980012178421
logged -1.0079126880541

uf 0.04158276319503784
logged -3.1800695438531847
uf 0.1871224343776703
logged -1.6759921470769106
uf 0.5052213668823242
logged -0.6827585954921579
-2.0368157309568127
uf 0.364980012178421
logged -1.0079126880541032
uf 1.6424100548028946
logged 0.49616470872217083
uf 0.21201999485492706
logged -1.5510746934041084
-1.0740776250455604
uf 0.364980012178421
logged -1.0079126880541032
uf 1.6424100548028946
logged 0.49616470872217083
uf 0.21201999485492706
logged -1.5510746934041084
-0.3036290342819481
uf 0.364980012178421
logged -1.0079126880541032
uf 1.6424100548028946
logged 0.49616470872217083
uf 0.21201999485492706
logged -1.5510746934041084
-0.9940013424267048
uf 0.364980012178421
logged -1.0079126880541032
uf 1.6424100548028946
logged 0.49616470872217083
uf 0.21201999485492706
logged -1.5510746934041084
-1.227274517469573
uf 0.364980012178421
logged -1.0079126880541032
uf 1.6424100548028946
logged 0.49616470872217083
uf 0.21201999485492706
logged -1.5510746934041084
-1.202900980867854

In [6]:
from IPython.display import IFrame
IFrame("../figures/resource_line_plot.pdf", width=600, height=280)

In [7]:
IFrame("../figures/resource_radar_plot.pdf", width=600, height=500)